In [0]:
spark.sql('use database Retail_DB_Second_Layer')
spark.sql("""create table if not exists silver_customers(
             customer_id integer,
             name string,
             email string,
             country string,
             customer_type string,
             registration_date date,
             age integer,
             gender string,
             total_purchases integer,
             last_updated_at timestamp
          )""")

DataFrame[]

In [0]:
#It's just to make sure of incremental data coming in
last_updated_df = spark.sql('select max(last_updated_at) as last_completed from silver_customers')
last_updated_time = last_updated_df.collect()[0]['last_completed']
if last_updated_time is None:
    last_updated_time = '1900-01-01T00:00:00.000+00:00'

In [0]:
#creating temporary view just to run in this case
spark.sql(f"""
          create or replace temporary view first_incremental as
          select * from Retail_DB_Initial.Bronze_customer as c where c.recived_at  > '{last_updated_time}' """)

DataFrame[]

In [0]:
spark.sql("select *  from first_incremental limit 10").show()

+-----------+-----------+--------------------+---------+-------------+-----------------+---+------+---------------+--------------------+
|customer_id|       name|               email|  country|customer_type|registration_date|age|gender|total_purchases|          recived_at|
+-----------+-----------+--------------------+---------+-------------+-----------------+---+------+---------------+--------------------+
|          1| Customer 1|customer1@example...|Australia|      Regular|       2011-05-15| 22|  Male|            191|2024-12-24 02:42:...|
|          2| Customer 2|customer2@example...|   France|      Premium|       2018-11-27| 52| Other|            145|2024-12-24 02:42:...|
|          3| Customer 3|customer3@example...|   Canada|      Premium|       2015-10-01| 32| Other|            691|2024-12-24 02:42:...|
|          4| Customer 4|customer4@example...|      USA|      Premium|       2011-01-19| 70| Other|            644|2024-12-24 02:42:...|
|          5| Customer 5|customer5@exampl

In [0]:
spark.sql("""
          create or replace temporary view silver_incremental as
          select
          customer_id,
          case 
            when name is not null then initcap(trim(name))
            else 'Unknown'
          end as name,
          case 
            when email is not null then lower(trim(email))
            else null
          end as email,
          case 
            when country is not null then trim(country)
            else 'Unknown'
          end as country,
          case 
            when customer_type in ('Regular','Premium','VIP') then customer_type
            else 'Unknown'
          end as customer_type,
          registration_date,
          case
            when age between 18 and 100 then age
            else null
          end as age,
          case
            when gender in ('Male','Female','Other') then gender
            else null
          end as gender,
          case 
            when total_purchases > 0  then total_purchases
            else 0
          end as total_purchases,
          current_timestamp as last_updated_at
          from first_incremental 
          where customer_id is not null and total_purchases>0 and email is not null""")

DataFrame[]

In [0]:
display(spark.sql("select * from silver_incremental limit 10"))

customer_id,name,email,country,customer_type,registration_date,age,gender,total_purchases,last_updated_at
1,Customer 1,customer1@example.com,Australia,Regular,2011-05-15,22,Male,191,2024-12-24T06:06:29.022Z
2,Customer 2,customer2@example.com,France,Premium,2018-11-27,52,Other,145,2024-12-24T06:06:29.022Z
3,Customer 3,customer3@example.com,Canada,Premium,2015-10-01,32,Other,691,2024-12-24T06:06:29.022Z
4,Customer 4,customer4@example.com,USA,Premium,2011-01-19,70,Other,644,2024-12-24T06:06:29.022Z
5,Customer 5,customer5@example.com,Germany,Regular,2021-08-26,66,Other,508,2024-12-24T06:06:29.022Z
6,Customer 6,customer6@example.com,France,Premium,2015-03-02,20,Male,704,2024-12-24T06:06:29.022Z
7,Customer 7,customer7@example.com,China,Premium,2018-05-24,24,Female,892,2024-12-24T06:06:29.022Z
8,Customer 8,customer8@example.com,China,Regular,2023-10-02,26,Male,488,2024-12-24T06:06:29.022Z
9,Customer 9,customer9@example.com,Japan,Premium,2014-10-05,36,Other,30,2024-12-24T06:06:29.022Z
10,Customer 10,customer10@example.com,Brazil,Premium,2017-08-30,30,Male,959,2024-12-24T06:06:29.022Z


In [0]:
spark.sql("""
    MERGE INTO silver_customers target
    USING silver_incremental source
    ON target.customer_id = source.customer_id
    WHEN MATCHED THEN 
        UPDATE SET 
            target.name = source.name,
            target.email = source.email,
            target.country = source.country,
            target.customer_type = source.customer_type,
            target.registration_date = source.registration_date,
            target.age = source.age,
            target.gender = source.gender,
            target.total_purchases = source.total_purchases,
            target.last_updated_at = source.last_updated_at
    WHEN NOT MATCHED THEN 
        INSERT (customer_id, name, email, country, customer_type, registration_date, age, gender, total_purchases, last_updated_at)
        VALUES (source.customer_id, source.name, source.email, source.country, source.customer_type,  source.registration_date, source.age, source.gender, source.total_purchases, source.last_updated_at)
""")


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("select count(*) from silver_customers").show()

+--------+
|count(*)|
+--------+
|    1000|
+--------+

